In [5]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import date, timedelta, datetime
from catboost import CatBoostRegressor, Pool,CatBoostClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn import metrics
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from sklearn import preprocessing
import time
from datetime import date, timedelta
from openpyxl import Workbook, load_workbook
import math

In [6]:
index = []

In [8]:
index2 = []
index3 = []

In [34]:

#month_end = int(input('Введите число конца месяца (например 31): '))
#name_file_save = 'test_1.txt'
# f = open('name_file_save', w+)
# f.write(значение)
# f.close()

start_time = time.time()
#WebPriceId     tDateObserve        tStockStatus        tCurrentPrice



chunksize = 10000 #батч
Id_corr = []
file_name = 'DS_test8(2020-06--2022-08-21).csv'
array = np.array([]) # Массив для
cor_array = np.array([]) #Массив CPI

id = 1 #Переменная которая содержит id признака которого сейчас рассматриваем
num=1
flag = False
#s = 0
count =0
mean_cpi = 0

#
def data_exel_read(filename_read):

    exel = load_workbook(filename_read) #загружаем ексель файл как объект
    sheet = exel.sheetnames #получаем в виде списка названия листов
    ws = exel[sheet[0]]
    data = ws.values
    Data = pd.DataFrame(data).to_numpy()
    return Data[2][1:]

date_df_y = data_exel_read('Y_train_adaptive.xlsx')
date_df_y = date_df_y.astype(float)




def processing(array, id):

    global cor_array, mean_cpi,count
    df = pd.DataFrame(data = array, columns= ['WebPriceId','tDateObserve','tStockStatus','tCurrentPrice'])
    array_options = df.to_numpy()
    OutOfStock = 0
    #print(df['tStockStatus'].shape)
    if df['tStockStatus'].value_counts()['InStock'] > 24:
        #dt  = datetime.strptime(array[1][1], '%Y-%m-%d %H:%M:%S.%f').date()
        df = df[df['tStockStatus']=='InStock']
        #df['tCurrentPrice'] = np.where((df.tStockStatus == 'OutOfStock'), True, df.tCurrentPrice)
        array = df.to_numpy()
        #print(array)


        first_inst  = datetime.strptime(array[0][1], '%Y-%m-%d %H:%M:%S.%f').date()
        second_inst = datetime(2022, 8, 31).date() #Важно!!! До какого заполняем



        date_range = pd.Series(pd.date_range(first_inst, periods = (second_inst-first_inst).days+1, freq ='D'))
        #print(array)
        zero_array = np.zeros(date_range.shape[0])

        data_train = pd.DataFrame({'Data': date_range, 'price': zero_array, 'Bool': zero_array})
        data_train['price'] = np.where((data_train.price == 0.0), None, data_train.price)
        data_train['Bool'] = np.where((data_train.Bool == 0.0), None, data_train.Bool)

        munth = pd.to_datetime(data_train.Data[0]).month
        day = pd.to_datetime(data_train.Data[0]).day

        for i in range(array.shape[0]):
            data_train['price'] = np.where((data_train.Data == pd.to_datetime(datetime.strptime(array[i][1], '%Y-%m-%d %H:%M:%S.%f').date())), array[i][3], data_train.price)
            #print(array[i][1])

        data_train = data_train.fillna(method='ffill') #Получаем датафрейм с ценами и датами
        num_day = 20 #''' !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! '''

        for i in range((datetime(2021, 5, 31).date()-first_inst).days,zero_array.shape[0]): #Не забыть изменить смотря от дня
            if  pd.to_datetime(data_train.Data[i]).day == num_day or ((pd.to_datetime(data_train.Data[i])+timedelta(days=1)).day == 1): #
                data_train['Bool'] = np.where((data_train.Data == pd.to_datetime(data_train.Data[i])), True, data_train.Bool)
                #print(pd.to_datetime(data_train.Data[i])+timedelta(days=1))

        data_train.dropna(subset=['Bool'], inplace=True)
        data_train = data_train.drop('Bool', axis=1)
        #print(data_train)
        #cpi = CPI(data_train,id)

        #cor_array = np.array([])
        dataframe_array = data_train.to_numpy()
        data_train['CPI'] = np.zeros(data_train.shape[0])
        #print(data_train)
        #dataframe = dataframe[['price','CPI']].to_numpy()
        #price_first = 0
        #price_second = 0
        CPI = np.array([])

        if ((dataframe_array[0][0]).date()).day == num_day: #Не забыть изменить смотря от дня
            dataframe_array = dataframe_array[1:]
            #print(dataframe_array)

        for i in range(0,data_train.shape[0]-1,2):
            #print(data_train.shape[0])
            CPI = np.append(CPI,(((dataframe_array[i+1][1]/dataframe_array[i][1])*100)-100))
        #print(CPI)
        #dataframe['CPI'] = np.where((dataframe.Data == pd.to_datetime((dataframe_array[i+2][0]).date())), ((dataframe_array[i+1][1]/dataframe_array[i][1])*100)-100, dataframe.CPI)

        #print(date_df_y[len(date_df_y)-12:])
        '''
        if CPI.shape[0] == 12 and CPI.sum()!=0 and (np.corrcoef(CPI, date_df_y[len(date_df_y)-CPI.shape[0]:])[0][1]) > 0.86:
            #cor_array = np.append(cor_array,(np.corrcoef(CPI, date_df_y[len(date_df_y)-12:])[0][1]))
            cor_array = np.append(cor_array,(np.array([CPI])))
            print('Корреляция равна = ',np.corrcoef(CPI, date_df_y[len(date_df_y)-CPI.shape[0]:])[0][1], 'id = ', id)
            print(CPI)
            mean_cpi += CPI[-1]
            count+=1
        '''

        #cor_array = np.append(cor_array,(np.corrcoef(CPI, date_df_y[len(date_df_y)-12:])[0][1]))
        cor_array = np.append(cor_array,(np.array([CPI])))
        print('Евклидово расстояние = ',math.dist(CPI,date_df_y[len(date_df_y)-CPI.shape[0]:]), 'id = ', id)
        print(CPI)
        mean_cpi += CPI[-1]
        count+=1

'''
        #return cpi
        #print(data_train.to_string())
        #print(array.shape[0])'''





for chunk in pd.read_csv(file_name, sep='\t', chunksize=chunksize):
    #print(2)
    '''
    if chunk['WebPriceId'].to_numpy()[0] == id:
        array = np.append(array, (chunk.to_numpy()))
    else:
        id = chunk['WebPriceId'].to_numpy()[0] #Обновляем айдишник
        #print(array)
        array = np.array([]) #Очищаем массив
        array = np.append(array, (chunk.to_numpy())) #добавляем первое значение нового айдишника
        print(id)'''
    #print(id in chunk['WebPriceId'].to_numpy())
    #sum+=chunk.to_numpy().shape[0]
    #print(chunk.to_string())
    while id in chunk['WebPriceId'].to_numpy():
        if id-1 in chunk['WebPriceId'].to_numpy() and flag == True:
            #print(np.reshape(array, (int(array.shape[0]/4), 4)))
            array = np.append(array, (chunk[chunk['WebPriceId'] == id-1].to_numpy())) #Добавляем значения
            array = np.reshape(array, (int(array.shape[0]/4), 4))
            # Тут чёт делаем и потом очищаем массив и берём следующий батч
            #cor_array = np.append(cor_array,(processing(array,id)))
            if id in index2: #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
                processing(array,id)
            #print(np.reshape(array, (int(array.shape[0]/4), 4)))
            array = np.array([]) #Очищаем массив
            id+=1
            flag = False
            #break

        elif id+1 in chunk['WebPriceId'].to_numpy(): #Если следующее значение есть то обнуляем значение и меняем id
            array = np.append(array, (chunk[chunk['WebPriceId'] == id].to_numpy()))
            # Тут чёт делаем и потом очищаем массив и берём следующий батч
            array = np.reshape(array, (int(array.shape[0]/4), 4))
            #print(pd.DataFrame(data = array, columns= ['WebPriceId','tDateObserve','tStockStatus','tCurrentPrice'])['tStockStatus'].value_counts())
            if id in index2: #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
                processing(array,id)
            array = np.array([]) #Очищаем массив
            id+=1
            flag = False
            #break

        elif id+1 not in chunk['WebPriceId'].to_numpy():#Если следующего значения нет
            array = np.append(array, (chunk[chunk['WebPriceId'] == id].to_numpy()))
            id+=1
            flag = True

    #print(id)
    #print(cor_array.shape[0])

    if id >= 50000:
        break


name_file_save = 'test_adp_8.txt'
f = open(name_file_save,'w+')
f.write(str(mean_cpi/count))
f.close()


Евклидово расстояние =  18.355632265654137 id =  1055
[-0.84033613  0.         -0.85470085  0.86956522  0.         -4.54545455
  5.51181102  0.          1.49253731 13.84615385  0.         -4.31654676
  0.          0.         -1.5037594 ]
Евклидово расстояние =  36.22910126221773 id =  1359
[ -1.50699677  -0.75431034  -1.73724213   6.53104925   6.48535565
  -0.66225166   0.83194676  -0.74257426  31.58326497   6.9669247
 -11.38157895  -0.07806401   0.          -1.08949416   0.15760441]
Евклидово расстояние =  13.933157897916407 id =  4343
[-1.61691542  0.          0.          0.          0.          0.
  0.         -1.89633375  0.78843627 10.03911343 -0.9478673   0.
  0.         -1.21065375 -0.73529412]
Евклидово расстояние =  20.053410220867452 id =  49458
[-0.97303308 -1.28205128  0.11293055 10.77010514  3.64699332  0.
 -0.95742001  2.1206766  -0.17356806 10.133579   -7.15181932 -1.05941962
  0.          0.30260708  0.6514658 ]


In [ ]:
######################

In [ ]:
#month_end = int(input('Введите число конца месяца (например 31): '))
#name_file_save = 'test_1.txt'
# f = open('name_file_save', w+)
# f.write(значение)
# f.close()

start_time = time.time()
#WebPriceId     tDateObserve        tStockStatus        tCurrentPrice



chunksize = 10000 #батч
Id_corr = []
file_name = 'DS_test2(2020-06--2022-06-21).csv'
array = np.array([]) # Массив для
cor_array = np.array([]) #Массив CPI

id = 1 #Переменная которая содержит id признака которого сейчас рассматриваем
num=1
flag = False
#s = 0
count =0
mean_cpi = 0

#
def data_exel_read(filename_read):

    exel = load_workbook(filename_read) #загружаем ексель файл как объект
    sheet = exel.sheetnames #получаем в виде списка названия листов
    ws = exel[sheet[0]]
    data = ws.values
    Data = pd.DataFrame(data).to_numpy()
    return Data[2][1:]

date_df_y = data_exel_read('Y_train_adaptive.xlsx')
date_df_y = date_df_y.astype(float)




def processing(array, id):

    global cor_array, mean_cpi,count
    df = pd.DataFrame(data = array, columns= ['WebPriceId','tDateObserve','tStockStatus','tCurrentPrice'])
    array_options = df.to_numpy()
    OutOfStock = 0
    #print(df['tStockStatus'].shape)
    if df['tStockStatus'].value_counts()['InStock'] > 24:
        #dt  = datetime.strptime(array[1][1], '%Y-%m-%d %H:%M:%S.%f').date()
        df = df[df['tStockStatus']=='InStock']
        #df['tCurrentPrice'] = np.where((df.tStockStatus == 'OutOfStock'), True, df.tCurrentPrice)
        array = df.to_numpy()
        #print(array)


        first_inst  = datetime.strptime(array[0][1], '%Y-%m-%d %H:%M:%S.%f').date()
        second_inst = datetime(2022, 5, 31).date() #Важно!!! До какого заполняем



        date_range = pd.Series(pd.date_range(first_inst, periods = (second_inst-first_inst).days+1, freq ='D'))
        #print(array)
        zero_array = np.zeros(date_range.shape[0])

        data_train = pd.DataFrame({'Data': date_range, 'price': zero_array, 'Bool': zero_array})
        data_train['price'] = np.where((data_train.price == 0.0), None, data_train.price)
        data_train['Bool'] = np.where((data_train.Bool == 0.0), None, data_train.Bool)

        munth = pd.to_datetime(data_train.Data[0]).month
        day = pd.to_datetime(data_train.Data[0]).day

        for i in range(array.shape[0]):
            data_train['price'] = np.where((data_train.Data == pd.to_datetime(datetime.strptime(array[i][1], '%Y-%m-%d %H:%M:%S.%f').date())), array[i][3], data_train.price)
            #print(array[i][1])

        data_train = data_train.fillna(method='ffill') #Получаем датафрейм с ценами и датами
        num_day = 20 #''' !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! '''

        for i in range((datetime(2021, 5, 31).date()-first_inst).days,zero_array.shape[0]): #Не забыть изменить смотря от дня
            if ((pd.to_datetime(data_train.Data[i])+timedelta(days=1)).day == 1): #pd.to_datetime(data_train.Data[i]).day == num_day or
                data_train['Bool'] = np.where((data_train.Data == pd.to_datetime(data_train.Data[i])), True, data_train.Bool)
                #print(pd.to_datetime(data_train.Data[i])+timedelta(days=1))

        data_train.dropna(subset=['Bool'], inplace=True)
        data_train = data_train.drop('Bool', axis=1)
        #print(data_train)
        #cpi = CPI(data_train,id)

        #cor_array = np.array([])
        dataframe_array = data_train.to_numpy()
        data_train['CPI'] = np.zeros(data_train.shape[0])
        #print(data_train)
        #dataframe = dataframe[['price','CPI']].to_numpy()
        #price_first = 0
        #price_second = 0
        CPI = np.array([])

        if ((dataframe_array[0][0]).date()).day == num_day: #Не забыть изменить смотря от дня
            dataframe_array = dataframe_array[1:]
            #print(dataframe_array)

        for i in range(0,data_train.shape[0]-1,2):
            #print(data_train.shape[0])
            CPI = np.append(CPI,(((dataframe_array[i+1][1]/dataframe_array[i][1])*100)-100))
        #print(CPI)
            #dataframe['CPI'] = np.where((dataframe.Data == pd.to_datetime((dataframe_array[i+2][0]).date())), ((dataframe_array[i+1][1]/dataframe_array[i][1])*100)-100, dataframe.CPI)

        #print(date_df_y[len(date_df_y)-12:])
        '''
        if CPI.shape[0] == 12 and CPI.sum()!=0 and (np.corrcoef(CPI, date_df_y[len(date_df_y)-CPI.shape[0]:])[0][1]) > 0.86:
            #cor_array = np.append(cor_array,(np.corrcoef(CPI, date_df_y[len(date_df_y)-12:])[0][1]))
            cor_array = np.append(cor_array,(np.array([CPI])))
            print('Корреляция равна = ',np.corrcoef(CPI, date_df_y[len(date_df_y)-CPI.shape[0]:])[0][1], 'id = ', id)
            print(CPI)
            mean_cpi += CPI[-1]
            count+=1
        '''
        if CPI.sum()!=0 and (math.dist(CPI,date_df_y[len(date_df_y)-CPI.shape[0]:])) < 4: #
            #cor_array = np.append(cor_array,(np.corrcoef(CPI, date_df_y[len(date_df_y)-12:])[0][1]))
            cor_array = np.append(cor_array,(np.array([CPI])))
            print('Евклидово расстояние = ',math.dist(CPI,date_df_y[len(date_df_y)-CPI.shape[0]:]), 'id = ', id)
            print(CPI)
            index3.append(id)
            mean_cpi += CPI[-1]
            count+=1

'''
        #return cpi
        #print(data_train.to_string())
        #print(array.shape[0])'''





for chunk in pd.read_csv(file_name, sep='\t', chunksize=chunksize):
    #print(2)
    '''
    if chunk['WebPriceId'].to_numpy()[0] == id:
        array = np.append(array, (chunk.to_numpy()))
    else:
        id = chunk['WebPriceId'].to_numpy()[0] #Обновляем айдишник
        #print(array)
        array = np.array([]) #Очищаем массив
        array = np.append(array, (chunk.to_numpy())) #добавляем первое значение нового айдишника
        print(id)'''
    #print(id in chunk['WebPriceId'].to_numpy())
    #sum+=chunk.to_numpy().shape[0]
    #print(chunk.to_string())
    while id in chunk['WebPriceId'].to_numpy():
        if id-1 in chunk['WebPriceId'].to_numpy() and flag == True:
            #print(np.reshape(array, (int(array.shape[0]/4), 4)))
            array = np.append(array, (chunk[chunk['WebPriceId'] == id-1].to_numpy())) #Добавляем значения
            array = np.reshape(array, (int(array.shape[0]/4), 4))
            # Тут чёт делаем и потом очищаем массив и берём следующий батч
            #cor_array = np.append(cor_array,(processing(array,id)))
            processing(array,id)
            #print(np.reshape(array, (int(array.shape[0]/4), 4)))
            array = np.array([]) #Очищаем массив
            id+=1
            flag = False
            #break

        elif id+1 in chunk['WebPriceId'].to_numpy(): #Если следующее значение есть то обнуляем значение и меняем id
            array = np.append(array, (chunk[chunk['WebPriceId'] == id].to_numpy()))
            # Тут чёт делаем и потом очищаем массив и берём следующий батч
            array = np.reshape(array, (int(array.shape[0]/4), 4))
            #print(pd.DataFrame(data = array, columns= ['WebPriceId','tDateObserve','tStockStatus','tCurrentPrice'])['tStockStatus'].value_counts())
            processing(array,id)
            array = np.array([]) #Очищаем массив
            id+=1
            flag = False
            #break

        elif id+1 not in chunk['WebPriceId'].to_numpy():#Если следующего значения нет
            array = np.append(array, (chunk[chunk['WebPriceId'] == id].to_numpy()))
            id+=1
            flag = True

    #print(id)
    #print(cor_array.shape[0])

    if id >= 30000:
        break
'''
    if cor_array.shape[0] == 48:
        print(cor_array)
        break
'''

'''
name_file_save = 'test_1.txt'
f = open(name_file_save,'w+')
f.write(str(mean_cpi/count))
f.close()'''
print(id)
print('Предсказание = :', mean_cpi/count)
# 800 сек рекорд!!! +- 13 мин
print("--- %s Время: ---" % (time.time() - start_time))

KeyboardInterrupt: 

In [23]:
print(index)

[1055, 1359, 4343, 49458, 1055, 1359, 4343, 49458]


In [24]:
index2 = index
print(index2)

[1055, 1359, 4343, 49458, 1055, 1359, 4343, 49458]


In [25]:
print(index3)

[8510, 8537, 8595, 10633, 14145, 16974, 17010, 18250, 19303, 22435]


In [95]:
a = [1.5]
b = [1.0]
print(math.dist(a,b))

0.5
